In [2]:
import tqdm
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import pandas as pd

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

r('''
    source('../../imputation2/notebooks/repos/ALRA/alra.R')
''')

In [1]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = np.log(df.loc[rows, cols] + 1)
    df_norm = df.copy()
    df_norm = scprep.normalize.library_size_normalize(df_norm)    
    df_norm = scprep.transform.sqrt(df_norm)
    X_norm = pd.DataFrame(df_norm, columns=df.columns)
    labels = df.index
    return X_norm, labels

In [3]:
import time

res = []
dir_list = ['baron', 'brosens', 'carey', 'hcabm40k', 'chang']
for i in tqdm(dir_list):
    X_norm, labels = get_data(i)
    start_time = time.time()
    r.assign("A", X_norm)
    r('''
        A = as.matrix(A)
    ''')
    r('''
        # Library and log normalize the data
        A_norm <- normalize_data(A)
        # Choose k. 
        k_choice <- choose_k(A_norm)
    ''')
    pred = r('''
        A_norm_completed <- alra(A_norm,k=k_choice$k)[[3]]
    ''')

    end_time = time.time()
    res.append(end_time - start_time)

  0%|          | 0/5 [00:00<?, ?it/s]/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


Read matrix with 1791 cells and 8669 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 21.79% nonzero to 78.61% nonzero


 20%|██        | 1/5 [00:46<03:06, 46.65s/it]/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


Read matrix with 3567 cells and 8102 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 26.43% nonzero to 80.15% nonzero


 40%|████      | 2/5 [02:15<03:35, 71.72s/it]/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


...

 60%|██████    | 3/5 [05:46<04:29, 134.91s/it]/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


Read matrix with 4064 cells and 5125 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 25.67% nonzero to 78.25% nonzero


 80%|████████  | 4/5 [07:50<02:10, 130.65s/it]/export/scratch/inoue019/envs/gpu1/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


Read matrix with 1471 cells and 1002 genes
Getting nonzeros
Randomized SVD
Find the 0.001000 quantile of each gene
Sweep
Scaling all except for 0 columns
0.00% of the values became negative in the scaling process and were set to zero
The matrix went from 44.83% nonzero to 76.40% nonzero


100%|██████████| 5/5 [08:49<00:00, 105.84s/it]


In [4]:
res

[38.12728714942932,
 62.40417408943176,
 131.48075437545776,
 38.40599274635315,
 5.904771566390991]